# Load New Multi-turn dialog
can be csv file, must have the following columns
- date
- dlg_id
- turn
- speaker
- text

In [1]:
import pandas as pd
from pathlib import Path
import torch
from transformers import pipeline

from pathlib import Path
import sys
cwd = Path.cwd()
common_code_path = cwd.parent.parent / "plp_common_code"

if str(common_code_path) not in sys.path:
    sys.path.append(str(common_code_path))
from amhere import foundme
print(foundme())
del foundme

from clf_Helper import load_hf_pdt_clf

cwd = Path.cwd()
data_input_path = cwd / 'data_input'

common code folder found.


In [2]:
csv_path_file = data_input_path / "Synth_400_banking_multi-turn_dlg.csv"
print(csv_path_file)
df = pd.read_csv(csv_path_file)
print(df.shape)
print(df["dlg_id"].nunique())
display(df.head(3))

D:\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP\06_Code_Actuals\3.4_Components_Deployed_Soln\3.4.7_Topics_DB\data_input\Synth_400_banking_multi-turn_dlg.csv
(2979, 5)
400


date               dlg_id  turn                     speaker  \
0  2024-10-23  2024-10-14__dlg_103     0                    Customer   
1  2024-10-23  2024-10-14__dlg_103     1  Customer Service Executive   
2  2024-10-23  2024-10-14__dlg_103     2                    Customer   

                                                text  
0  Hi, I’m really frustrated with my current expe...  
1  I’m sorry to hear that you’re unhappy. Please ...  
2  Honestly, I've been facing a lot of issues wit...

# Data Preprocessing
- get only the customer utterances

In [3]:
# retain only customer utterances
fc1 = df['speaker'] == 'Customer'
fc2 = df['speaker'] == 'customer'
dft = df.loc[fc1 | fc2, :].copy()
dft.reset_index(drop=True, inplace=True)

dft.head()

date                dlg_id  turn   speaker  \
0  2024-10-23   2024-10-14__dlg_103     0  Customer   
1  2024-10-23   2024-10-14__dlg_103     2  Customer   
2  2024-10-23   2024-10-14__dlg_103     4  Customer   
3  2024-10-23  2024-10-14__dlg_1134     1  Customer   
4  2024-10-23  2024-10-14__dlg_1134     3  Customer   

                                                text  
0  Hi, I’m really frustrated with my current expe...  
1  Honestly, I've been facing a lot of issues wit...  
2  I appreciate the offer, but this has been an o...  
3  Hi, Jordan! I'm Alice. I recently received my ...  
4  Certainly! There’s a transaction dated Septemb...

# Drop unneeded columns - "turn", "speaker"

In [4]:
dft = dft[['date', 'dlg_id', 'text']]
dft.head()

date                dlg_id  \
0  2024-10-23   2024-10-14__dlg_103   
1  2024-10-23   2024-10-14__dlg_103   
2  2024-10-23   2024-10-14__dlg_103   
3  2024-10-23  2024-10-14__dlg_1134   
4  2024-10-23  2024-10-14__dlg_1134   

                                                text  
0  Hi, I’m really frustrated with my current expe...  
1  Honestly, I've been facing a lot of issues wit...  
2  I appreciate the offer, but this has been an o...  
3  Hi, Jordan! I'm Alice. I recently received my ...  
4  Certainly! There’s a transaction dated Septemb...

# Load Classifiers

In [5]:
pdt_clf = load_hf_pdt_clf(
    nlp_task='text-classification', 
    hf_model_chkpt="richardchai/plp_pdt_clr_tinybert")

action_clf = load_hf_pdt_clf(
    nlp_task='text-classification', 
    hf_model_chkpt="richardchai/plp_action_clr_distilbert")

sentiment_clf = load_hf_pdt_clf(
    nlp_task='text-classification', 
    hf_model_chkpt="richardchai/plp_sentiment_clr_distilbert")

# Classify Product from Customer Utterance

In [6]:
del df

In [7]:
def predict_product(x: [str, list]):
    response = pdt_clf(x)
    product = [r['label'] for r in response]
    if isinstance(product, list) and len(product) < 2:
        return product[0]
    else:
        return str(product)[1:-1]
    
dft['product'] = dft['text'].apply(lambda x: predict_product(x))
display(dft.head())
display(dft.tail())

date                dlg_id  \
0  2024-10-23   2024-10-14__dlg_103   
1  2024-10-23   2024-10-14__dlg_103   
2  2024-10-23   2024-10-14__dlg_103   
3  2024-10-23  2024-10-14__dlg_1134   
4  2024-10-23  2024-10-14__dlg_1134   

                                                text          product  
0  Hi, I’m really frustrated with my current expe...  current_account  
1  Honestly, I've been facing a lot of issues wit...  current_account  
2  I appreciate the offer, but this has been an o...  current_account  
3  Hi, Jordan! I'm Alice. I recently received my ...  current_account  
4  Certainly! There’s a transaction dated Septemb...       debit_card

date               dlg_id  \
1399  2024-10-23  2024-10-14__dlg_873   
1400  2024-10-23  2024-10-14__dlg_873   
1401  2024-10-23  2024-10-14__dlg_958   
1402  2024-10-23  2024-10-14__dlg_958   
1403  2024-10-23  2024-10-14__dlg_958   

                                                   text          product  
1399  Definitely. It’s clear that a lot of thought h...             loan  
1400  Sure thing; I’ll keep that in mind. Thank you ...          account  
1401  Hi, I'm really frustrated with the service I'v...          website  
1402  The problems have been happening consistently ...  current_account  
1403  I've tried that already, but there's been no i...          website

In [8]:
def predict_action(x: [str, list]):
    response = action_clf(x)
    action = [r['label'] for r in response]
    
    if isinstance(action, list) and len(action) < 2:
        return action[0]
    else:
        return str(action)[1:-1]
    
dft['action'] = dft['text'].apply(lambda x: predict_action(x))

In [9]:
def predict_sentiment(x: [str, list]):
    response = sentiment_clf(x)
    sentiment = [r['label'] for r in response]
    
    if isinstance(sentiment, list) and len(sentiment) < 2:
        return sentiment[0]
    else:
        return str(sentiment)[1:-1]
    
dft['sentiment'] = dft['text'].apply(lambda x: predict_sentiment(x))

In [10]:
dft.head()

date                dlg_id  \
0  2024-10-23   2024-10-14__dlg_103   
1  2024-10-23   2024-10-14__dlg_103   
2  2024-10-23   2024-10-14__dlg_103   
3  2024-10-23  2024-10-14__dlg_1134   
4  2024-10-23  2024-10-14__dlg_1134   

                                                text          product  \
0  Hi, I’m really frustrated with my current expe...  current_account   
1  Honestly, I've been facing a lot of issues wit...  current_account   
2  I appreciate the offer, but this has been an o...  current_account   
3  Hi, Jordan! I'm Alice. I recently received my ...  current_account   
4  Certainly! There’s a transaction dated Septemb...       debit_card   

     action      sentiment  
0  transfer        neutral  
1  transfer       negative  
2  transfer        neutral  
3   dispute  very negative  
4   dispute  very negative

In [11]:
dft.tail()

date               dlg_id  \
1399  2024-10-23  2024-10-14__dlg_873   
1400  2024-10-23  2024-10-14__dlg_873   
1401  2024-10-23  2024-10-14__dlg_958   
1402  2024-10-23  2024-10-14__dlg_958   
1403  2024-10-23  2024-10-14__dlg_958   

                                                   text          product  \
1399  Definitely. It’s clear that a lot of thought h...             loan   
1400  Sure thing; I’ll keep that in mind. Thank you ...          account   
1401  Hi, I'm really frustrated with the service I'v...          website   
1402  The problems have been happening consistently ...  current_account   
1403  I've tried that already, but there's been no i...          website   

        action      sentiment  
1399   unknown       negative  
1400    report  very negative  
1401    access       negative  
1402     close        neutral  
1403  exchange       negative

# Create Aspect Topic column

In [12]:
dft['aspect_topic'] = dft['action'] + '__' + dft['product']
dft.head()

date                dlg_id  \
0  2024-10-23   2024-10-14__dlg_103   
1  2024-10-23   2024-10-14__dlg_103   
2  2024-10-23   2024-10-14__dlg_103   
3  2024-10-23  2024-10-14__dlg_1134   
4  2024-10-23  2024-10-14__dlg_1134   

                                                text          product  \
0  Hi, I’m really frustrated with my current expe...  current_account   
1  Honestly, I've been facing a lot of issues wit...  current_account   
2  I appreciate the offer, but this has been an o...  current_account   
3  Hi, Jordan! I'm Alice. I recently received my ...  current_account   
4  Certainly! There’s a transaction dated Septemb...       debit_card   

     action      sentiment               aspect_topic  
0  transfer        neutral  transfer__current_account  
1  transfer       negative  transfer__current_account  
2  transfer        neutral  transfer__current_account  
3   dispute  very negative   dispute__current_account  
4   dispute  very negative        dispute__debit_card

# Append to Topics Database (using parquet to simulate for purpose of POC)

In [13]:
# topics_db.parquet
temp_df = pd.read_parquet("database/topics_db.parquet")
print(temp_df.shape)
display(temp_df.head())

(15378, 7)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment   aspect_topic  
0   neutral  inquire__card  
1   neutral  inquire__card  
2   neutral  inquire__card  
3  positive  inquire__card  
4  positive  inquire__card

In [14]:
print(dft.shape)
display(dft.head())

(1404, 7)


date                dlg_id  \
0  2024-10-23   2024-10-14__dlg_103   
1  2024-10-23   2024-10-14__dlg_103   
2  2024-10-23   2024-10-14__dlg_103   
3  2024-10-23  2024-10-14__dlg_1134   
4  2024-10-23  2024-10-14__dlg_1134   

                                                text          product  \
0  Hi, I’m really frustrated with my current expe...  current_account   
1  Honestly, I've been facing a lot of issues wit...  current_account   
2  I appreciate the offer, but this has been an o...  current_account   
3  Hi, Jordan! I'm Alice. I recently received my ...  current_account   
4  Certainly! There’s a transaction dated Septemb...       debit_card   

     action      sentiment               aspect_topic  
0  transfer        neutral  transfer__current_account  
1  transfer       negative  transfer__current_account  
2  transfer        neutral  transfer__current_account  
3   dispute  very negative   dispute__current_account  
4   dispute  very negative        dispute__debit_card

In [15]:
topics_db = pd.concat([temp_df, dft])
topics_db.reset_index(drop=True, inplace=True)
print(topics_db.shape)
display(topics_db.head())
display(topics_db.tail())

(16782, 7)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment   aspect_topic  
0   neutral  inquire__card  
1   neutral  inquire__card  
2   neutral  inquire__card  
3  positive  inquire__card  
4  positive  inquire__card

date               dlg_id  \
16777  2024-10-23  2024-10-14__dlg_873   
16778  2024-10-23  2024-10-14__dlg_873   
16779  2024-10-23  2024-10-14__dlg_958   
16780  2024-10-23  2024-10-14__dlg_958   
16781  2024-10-23  2024-10-14__dlg_958   

                                                    text          product  \
16777  Definitely. It’s clear that a lot of thought h...             loan   
16778  Sure thing; I’ll keep that in mind. Thank you ...          account   
16779  Hi, I'm really frustrated with the service I'v...          website   
16780  The problems have been happening consistently ...  current_account   
16781  I've tried that already, but there's been no i...          website   

         action      sentiment            aspect_topic  
16777   unknown       negative           unknown__loan  
16778    report  very negative         report__account  
16779    access       negative         access__website  
16780     close        neutral  close__current_account  
16781  exchange       negative       exchange__website

In [16]:
topics_db.to_parquet('database/topics_db.parquet', index=False)
topics_db.to_csv('database/topics_db.csv', index=False, encoding='utf-8-sig')